# Programming Project 1: Exhaustive Key Search

## By:
### Group 4
### Shashidhar Kalapatapu – 11545863
### Dharmateja Kollipara - 11555465


## Prerequisite Libraries

In [1]:
!pip3 install pycryptodomex
!pip3 install tqdm

You should consider upgrading via the 'C:\Users\admin\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\admin\anaconda3\python.exe -m pip install --upgrade pip' command.


### Description of Libraries
1. pycryptodomex has the implementation of AES-128
2. tqdm is a library which displays a visual progress bar of a loop running many iterations. 

## Imports

In [20]:
from Cryptodome.Cipher import AES
from tqdm import tqdm

## Utility functions 

In [21]:
# This is the encryption function. It gets a plaintext a key and returns the ciphertext and nonce.
def encryptor_CTR(message, key, nonce = None):
    cipher = AES.new(key, AES.MODE_CTR)
    ct = cipher.encrypt(message)    
    nonce = cipher.nonce
    return nonce, ct


# This is the decryption funtion. It gets a ciphertext, a nonce and a key and returns the plaintext.
def decryptor_CTR(ctxt, nonce, key):
    try:
        cipher = AES.new(key, AES.MODE_CTR, nonce=nonce)
        pt = cipher.decrypt(ctxt)
        return pt
    except ValueError as KeyError:
        return None


# Converting string to bytes.
def string_to_bytes(string_value):
    return bytearray(string_value,
                     encoding='utf-8')

def read_file(fn):
    f = open(fn, "r")
    value = f.read()
    f.close()
    return value

def read_bytes(fn):
    f = open(fn, "rb")
    value = f.read()
    f.close()
    return value

    
def write_file(fn, value):
    f = open(fn, "w")
    f.write(value)
    f.close()

def write_bytes(fn, value):
    f = open(fn, "wb")
    f.write(value)
    f.close()


def bitstring_to_bytes(s):
    v = int(s, 2)
    b = bytearray()

    while v:
        b.append(v & 0xff)
        v >>= 8
    return bytes(b[::-1])

## AES-128 Exhaustive Key Search Function

In [22]:
def exhaustiveKeySearch(files, keyPrefix, lastXBytes):
    lengthPostfix = lastXBytes * 8
    keySpace = 2 ** lengthPostfix
    print("EXHAUSTIVE KEY SEARCH OF AES-128")
    print("--------------------------------")
    print()
    print("The First 13 bytes of Hexadecimal Key are fixed.")
    
    print("80 00 00 00 00 00 00 00 00 00 00 00 00 XX XX XX")
    print("For the last 3 bytes, all combinations are searched from 00 to FF")
    print()
    print("Brute Force searching through ", keySpace, "possibilities ...")
    for i in tqdm(range(keySpace), desc='Progress '):
        # Define our hex key as a 'bytes' object
        key = b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' + i.to_bytes(lastXBytes, byteorder="big")

        # Construct and Initialize three AES-128 Ciphers in CTR Mode using the given three nonce bin files.
        c1 = AES.new(key, AES.MODE_CTR, nonce=files[1]['nonce'])
        c2 = AES.new(key, AES.MODE_CTR, nonce=files[2]['nonce'])
        c3 = AES.new(key, AES.MODE_CTR, nonce=files[3]['nonce'])

        # Using the defined AES cipher objects, perform decryptions on our given cipher texts
        d1 = c1.decrypt(files[1]['ctxt'])
        d2 = c2.decrypt(files[2]['ctxt'])
        d3 = c3.decrypt(files[3]['ctxt'])

        # Condition to check if all decryptions are matching with their respective plain texts
        if d1 == files[1]['txt'] and d2 == files[2]['txt'] and d3 == files[3]['txt']:
            print("Key found (in HEXADECIMAL):", key.hex())
            
            # We write this key to a bin file and txt files
            write_bytes(fn = path + "key.bin", value = key)
            write_file(fn = path + "key.txt", value = key.hex())
            
            break
    print("Key Written to the path: ", path + "key.bin")

In [23]:
# Driver Code. Here the execution starts
if __name__ == "__main__":
    # PATH of the folder where the bin and txt files should be set here
    path = 'D:/UNT Class Work/Spring 2023/CSCE 5050 Applications of Cryptography/Programming Project 1/'
    files = {
        1: {
            'ctxt': read_bytes(path + "c1.bin"),
            'nonce': read_bytes(path + "nonce1.bin"),
            'txt': read_file(path + "m1.txt").encode()
        },

        2: {
            'ctxt': read_bytes(path + "c2.bin"),
            'nonce': read_bytes(path + "nonce2.bin"),
            'txt': read_file(path + "m2.txt").encode()
        },

        3: {
            'ctxt': read_bytes(path + "c3.bin"),
            'nonce': read_bytes(path + "nonce3.bin"),
            'txt': read_file(path + "m3.txt").encode()
        },
        "challenge": {
            'ctxt': read_bytes(path + "c_c.bin"),
            'nonce': read_bytes(path + "nonce_c.bin"),
        }
    }
    
    # The first 13 bytes keyprefix which was already given in the problem question
    keyPrefix = b'\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
    
    #The last 3 bytes of our key are arbitrary. We need to find these 3 bytes
    lastXBytes = 3
    
    #Start our exhaustive search
    exhaustiveKeySearch(files, keyPrefix, lastXBytes)
    
    #Above code will run the search and write the key to a bin and text file.

EXHAUSTIVE KEY SEARCH OF AES-128
--------------------------------

The First 13 bytes of Hexadecimal Key are fixed.
80 00 00 00 00 00 00 00 00 00 00 00 00 XX XX XX
For the last 3 bytes, all combinations are searched from 00 to FF

Brute Force searching through  16777216 possibilities ...


Progress :  37%|█████████████████████▎                                   | 6286948/16777216 [06:01<10:02, 17407.51it/s]

Key found (in HEXADECIMAL): 800000000000000000000000005fee64
Key Written to the path:  D:/UNT Class Work/Spring 2023/CSCE 5050 Applications of Cryptography/Programming Project 1/key.bin


## Short Program to Decrypt the Challenge Cipher Text

In [25]:
# Program to Decrypt the Challenge Cipher Text 
print("Reading Key from the path: \n", path + "key.bin")
print()
key = read_bytes(fn = path + "key.bin")

print("Reading Challenge Nonce from the path: \n", path + "nonce_c.bin")
print()

print("Reading Challenge Cipher Text from the path: \n", path + "c_c.bin")
print()

print("Decrypting Challenge plaintext using the key ... ")
print()

challengeCipher = AES.new(key, AES.MODE_CTR, nonce=files["challenge"]['nonce'])
challengePlaintext = challengeCipher.decrypt(files["challenge"]['ctxt'])

print("Decrypted Challenge Plaintext: \n", challengePlaintext)
print()

print("Writing Challenge Plaintext file to this path ...\n", path + "c_m.txt")
write_file(fn = path + "c_m.txt", value = str(challengePlaintext))

Reading Key from the path: 
 D:/UNT Class Work/Spring 2023/CSCE 5050 Applications of Cryptography/Programming Project 1/key.bin

Reading Challenge Nonce from the path: 
 D:/UNT Class Work/Spring 2023/CSCE 5050 Applications of Cryptography/Programming Project 1/nonce_c.bin

Reading Challenge Cipher Text from the path: 
 D:/UNT Class Work/Spring 2023/CSCE 5050 Applications of Cryptography/Programming Project 1/c_c.bin

Decrypting Challenge plaintext using the key ... 

Decrypted Challenge Plaintext: 
 b'UNT is a community of dreamers and doers.'

Writing Challenge Plaintext file to this path ...
 D:/UNT Class Work/Spring 2023/CSCE 5050 Applications of Cryptography/Programming Project 1/c_m.txt
